In [1]:
from utils import experiment
import os

os.environ["OPENAI_API_KEY"] = ''

experiment.experiment(experiment_name="dw_ex_1", version="1.0")

/home/dongwoo/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


실험 설정 파일을 불러옵니다: {'version': '1.0', 'test_name': 'dw_ex_1', 'embedding_model': 'text-embedding-3-small', 'llm_model': 'gpt-4.1-mini', 'chunk_size': 300, 'chunk_overlap': 30, 'top_k': 3}
청킹 문서를 로드 완료하였습니다. 문서 개수: 37350


/home/dongwoo/codeit_mid_project/utils/experiment.py:126: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model=embedding_model_name)


임베딩 모델을 설정하였습니다: text-embedding-3-small
FAISS 인덱스를 불러왔습니다
retriever를 생성하였습니다. Top K: 3
RAG Chain을 생성하였습니다. LLM 모델: gpt-4.1-mini
질의 테스트를 시작합니다


/home/dongwoo/codeit_mid_project/rag_chain/RagChain_temp.py:24: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model_name=model_name, temperature=0.3)


질의를 입력하세요 (종료하려면 'exit' 입력):  서울과 관련된 제안요청서의 개요를 알고 싶어



Q. 서울과 관련된 제안요청서의 개요를 알고 싶어

A. 서울시청 공간정보담당관에서 발주하는 제안요청서는 서울시청 서소문별관 1동 3층에서 진행됩니다. 제안서 제출 시에는 입찰참가 공문(대표자 인감 날인), 입찰참가신청서, 가격입찰서 및 가격제안서(별지서식 6, 7)를 밀봉하여 제출해야 하며, 제안서 원본 1부와 평가본 및 평가본 요약서 각 9부 등 총 10부를 제출해야 합니다. 제안서에는 회사명, 회사 로고, 대표자 성명이 포함되어야 하며, 발표자료도 제안서 제출 시 함께 제출해야 합니다. 이 제안요청서는 서울시 공간정보와 관련된 사업을 대상으로 하고 있습니다.

응답이 저장되었습니다.


질의를 입력하세요 (종료하려면 'exit' 입력):  exit


질의 테스트를 종료합니다.


In [3]:
import sys
sys.path.append('./utils')  # evaluate.py가 utils 폴더에 있다면

import evaluate

# API 키 설정
evaluate.set_api_key("")

# 평가 진행 (경로만 지정)
evaluate.evaluate_from_jsonl("./experiment/dw_ex_1/dw_ex_1.jsonl")



===== 질의 1 평가 중 =====


AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}